In [1]:
import json
import os
import math
import librosa
from tensorflow.keras.models import load_model
import numpy as np
import noisereduce as nr

c:\Users\Ricky\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### code

In [2]:
model = load_model("./model/custom_cnn_0.0031.h5")
SAMPLE_RATE = 22050
TRACK_DURATION = 30  # second
SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION
# file_path = "../audio_tester/armada - hargai aku-[AudioTrimmer.com].mp3"
# file_path = "../../../Musics/2022+/SZA - Kill Bill (Audio).mp3"
# file_path = "../../../Musics/gama - satu ataw dua.mp3"
# file_path = "../../../Musics/2020s/Ava Max - Salt (Lyrics).mp3"
# file_path = "../../../Musics/2020s/Qorygore - Sampurasun (Lyrics) ft. Pretty Rico.mp3"
# file_path = "../../../Musics/2020s/Dj Quads - Punch.mp3"
file_path = "../mfcc_genre_detection/datasets/jazz/jazz.00033.wav"

# file_path = "soundfile.wav"

def splitsongs(X,window=0.05, overlap=0.5):
    # Empty lists to hold our results
    temp_X = []

    # Get the input song array size
    xshape = X.shape[0]
    chunk = int(xshape * window)
    offset = int(chunk * (1.0 - overlap))

    # Split the song and create new ones on windows
    spsong = [X[i : i + chunk] for i in range(0, xshape - chunk + offset, offset)]
    for s in spsong:
        if s.shape[0] != chunk:
            continue

        temp_X.append(s)

    return np.array(temp_X)

def to_melspectrogram(songs, n_fft=1024, hop_length=256):
    # Transformation function
    melspec = lambda x: librosa.feature.melspectrogram(
        y=x, sr=22050, n_fft=n_fft, hop_length=hop_length, n_mels=128
    )[
        :, :, np.newaxis
    ]  # keep n_mels=128. other values are for experimenting

    # map transformation of input songs to melspectrogram using log-scale
    tsongs = map(melspec, songs)
    # np.array([librosa.power_to_db(s, ref=np.max) for s in list(tsongs)])
    return np.array(list(tsongs))


sample_rate = 22000
signal, sr = librosa.load(file_path, sr=sample_rate)

# start_signal = (sample_rate * 60)
# end_signal = (sample_rate * 90)
# signal = signal[start_signal:end_signal]
signal = signal[:sample_rate * 30]

# signal = nr.reduce_noise(y=signal, sr=sample_rate)
print(signal.shape)

signals = splitsongs(signal, window=0.05)
x = to_melspectrogram(signals)
# x = np.array(x[0])
# x = x[..., np.newaxis]
# x = x[np.newaxis,...]
print(x.shape)
# x = x.reshape(1, 128, 129, 1)
# print(x)
prediction = model.predict(x)
# print(prediction.shape)


genres = {
    "Blues": 0,
    "Classical": 0,
    "Country": 0,
    "Disco": 0,
    "HipHop": 0,
    "Jazz": 0,
    "Metal": 0,
    "Pop": 0,
    "Reggae": 0,
    "Rock": 0,
}
genres_list = list(genres.keys())

for i in range(prediction.shape[0]):
    for idx, v in enumerate(prediction[i]):
        # print('val', genres['Reggae'])
        genres[genres_list[idx]] = ((genres[genres_list[idx]] * 3 )+ v) / 4
        # genres[genres_list[idx]] = genres[genres_list[idx]]+ v 


print("<<========= Genre Detection Accuracy =======>>")
total_val = 0
for genre in genres:
    print("{}\t\t==> {}%".format(genre, round(genres[genre] * 100, 2)))
    total_val = total_val +  round(genres[genre] * 100, 2)

print('total value',total_val)


(660000,)
(39, 128, 129, 1)
2/2 [==============================] - 1s 33ms/step
<<========= Genre Detection Accuracy =======>>
Blues		==> 10.36%
Classical		==> 3.03%
Country		==> 4.85%
Disco		==> 0.06%
HipHop		==> 0.15%
Jazz		==> 79.31%
Metal		==> 0.04%
Pop		==> 0.8%
Reggae		==> 0.84%
Rock		==> 0.55%
total value 99.99


### Testing

In [20]:
# print('accuracy ==>', accuracy)
# print('min accuracy  ==>', min_acc)
print("sum_accuracy ==>", sum_accuracy)

print("<<========= Genre Detection Accuracy =======>>")
# print(genres)
total_val = 0
for genre in genres:
    print("{}\t\t==> {}%".format(genre, round(genres[genre] * 10, 2)))
    total_val = total_val + genres[genre]

print("total_val ==>", total_val)

# print(
#     "mfcc_val ==> {} \nprediction ==> {}\npredicted_index ==> {}".format(
#         val.shape, prediction, predicted_index
#     )
# )
# for v in prediction:
#     print(v)
#     print('==='*30)
#     print(np.max(v))
#     print('==='*30)

sum_accuracy ==> 8.012447115033865
<<========= Genre Detection Accuracy =======>>
{'Blues': 0.01791323409997858, 'Classical': 0.5549905630796275, 'Country': 0.9404911054916738, 'Disco': 0.006302460954429989, 'HipHop': 0.0048047398280459674, 'Jazz': 8.012447115033865, 'Metal': 0.012815086361115391, 'Pop': 0.11817820123951606, 'Reggae': 0.004754828957061363, 'Rock': 0.3273026292445138}
Blues		==> 0.18%
Classical		==> 5.55%
Country		==> 9.4%
Disco		==> 0.06%
HipHop		==> 0.05%
Jazz		==> 80.12%
Metal		==> 0.13%
Pop		==> 1.18%
Reggae		==> 0.05%
Rock		==> 3.27%
total_val ==> 9.999999964289827


In [26]:
nilai = np.arange(0, 10)
genres = {
    "Blues": 0,
    "Classical": 0,
    "Country": 0,
    "Disco": 0,
    "HipHop": 0,
    "Jazz": 0,
    "Metal": 0,
    "Pop": 0,
    "Reggae": 0,
    "Rock": 0,
}

# genre_index = 0
# for i in range(nilai.size):
#     current_genre = list(genres.keys())[genre_index]
#     genres[current_genre] = i
#     genre_index += 1

for key, val in enumerate(genres):
    print(key)
    print(val)

print(math.ceil(661794 / 33000))

Blues
Classical
Country
Disco
HipHop
Jazz
Metal
Pop
Reggae
Rock

0
Blues
1
Classical
2
Country
3
Disco
4
HipHop
5
Jazz
6
Metal
7
Pop
8
Reggae
9
Rock
21
